In [30]:
import pandas as pd
import numpy as np
from pprint import pprint
dataset = pd.read_excel(r'C:\Users\Lenovo\Dropbox\My PC (DESKTOP-BHVANOA)\Desktop\data2.xlsx'
        ,names=['department','age','salary','class',])



dataset

,department,age,salary,class
0,sales,31-35,46k-50k,senior
1,sales,26-30,26k-30k,junior
2,sales,31-35,31k-35k,junior
3,systems,21-25,46k-50k,junior
4,systems,31-35,66k-70k,senior
...,...,...,...,...
160,secretary,26-30,26k-30k,junior
161,secretary,26-30,26k-30k,junior
162,secretary,26-30,26k-30k,junior
163,secretary,26-30,26k-30k,junior


# Calculate the entropy of a dataset

In [31]:
def entropy(target_col):
    """
    Calculate the entropy of a dataset.
    The only parameter of this function is the target_col parameter which specifies the target column
    """
    elements,counts = np.unique(target_col,return_counts = True)
    entropy = np.sum([(-counts[i]/np.sum(counts))*np.log2(counts[i]/np.sum(counts)) for i in range(len(elements))])
    return entropy

# Calculate the information gain

In [32]:
def InfoGain(data,split_attribute_name,target_name="class"):
   
    #Calculate the entropy of the total dataset
    total_entropy = entropy(data[target_name])
    
    ##Calculate the entropy of the dataset
    
    #Calculate the values and the corresponding counts for the split attribute 
    vals,counts= np.unique(data[split_attribute_name],return_counts=True)
    
    #Calculate the weighted entropy
    Weighted_Entropy = np.sum([(counts[i]/np.sum(counts))*entropy(data.where(data[split_attribute_name]==vals[i]).dropna()[target_name]) for i in range(len(vals))])
    
    Information_Gain = total_entropy - Weighted_Entropy
    return Information_Gain

# Create the tree structure

In [33]:
def ID3(data,originaldata,features,target_attribute_name="class",parent_node_class = None):

    if len(np.unique(data[target_attribute_name])) <= 1:
        return np.unique(data[target_attribute_name])[0]
    
    
    elif len(data)==0:
        return np.unique(originaldata[target_attribute_name])[np.argmax(np.unique(originaldata[target_attribute_name],return_counts=True)[1])]
    
    
    elif len(features) ==0:
        return parent_node_class
    
    
    else:
        
        parent_node_class = np.unique(data[target_attribute_name])[np.argmax(np.unique(data[target_attribute_name],return_counts=True)[1])]
        
        #Select the feature which best splits the dataset
        item_values = [InfoGain(data,feature,target_attribute_name) for feature in features] #Return the information gain values for the features in the dataset
        best_feature_index = np.argmax(item_values)
        best_feature = features[best_feature_index]
        
        #Create the tree structure. 
        tree = {best_feature:{}}
        
        
        features = [i for i in features if i != best_feature]
        
        
        for value in np.unique(data[best_feature]):
            value = value
        
            sub_data = data.where(data[best_feature] == value).dropna()
            
            subtree = ID3(sub_data,dataset,features,target_attribute_name,parent_node_class)
            
            tree[best_feature][value] = subtree
            
        return(tree)

In [34]:
def predict(query,tree,default = 1):
    
    for key in list(query.keys()):
        if key in list(tree.keys()):
            
            try:
                result = tree[key][query[key]] 
            except:
                return default
  
            
            result = tree[key][query[key]]
            
            if isinstance(result,dict):
                return predict(query,result)

            else:
                return result

# Train Test

In [35]:
def train_test_split(dataset):
    training_data = dataset.iloc[:10].reset_index(drop=True)
    testing_data = dataset.iloc[10:].reset_index(drop=True)
    return training_data,testing_data

training_data = train_test_split(dataset)[0]
testing_data = train_test_split(dataset)[1] 



def test(data,tree):
    queries = data.iloc[:,:-1].to_dict(orient = "records")
    
    #Create a empty DataFrame in whose columns the prediction of the tree are stored
    predicted = pd.DataFrame(columns=["predicted"]) 
    
    #Calculate the prediction accuracy
    for i in range(len(data)):
        predicted.loc[i,"predicted"] = predict(queries[i],tree,1.0) 
    print('The prediction accuracy is: ',(np.sum(predicted["predicted"] == data["class"])/len(data))*100,'%')

In [36]:
tree = ID3(training_data,training_data,training_data.columns[:-1])
pprint(tree)
test(testing_data,tree)

{'age': {'21-25': 'junior',
         '26-30': 'junior',
         '31-35': {'salary': {'31k-35k': 'junior',
                              '41k-45k': 'junior',
                              '46k-50k': 'senior',
                              '66k-70k': 'senior'}},
         '36-40': 'senior',
         '41-45': 'senior',
         '46-50': 'senior'}}
The prediction accuracy is:  100.0 %
